#**1 Introduccion**
En el siguiente cuaderno se realiza el codigo para invertir un vector, es decir:
<center>$Dado A=(a1,a2,a3,a4,a5)$</center>
<center>$El resultado es A´(a5,a4,a3,a2,a1)$</center>

Para el cual se pide por parametro la cantidad de elementos para vector a invertir. 

En cuanto a la paralelizacion[1]:

Se define la memoria de los vectores en CPU, se debe reeservar espacio de memoria GPU para los vectores a usar y luego se debe realizar la transferencia de datos de CPU a GPU. Luego, se debe definir la función kernel que ejecutará en GPU, dentro de esta funcion, para acceder al indice de la malla y al indice del hilo dentro del bloque se debe emplear: **blockIdx.x** y **threadIdx.x**;
Luego se calcula los índices globales donde cada hilo tenga que trabajar
en un área de datos diferente según la partición:
**int Idx = blockIdx.x * blockDim.x + threadIdx.x;** 
Genero la funcion *kernel* y luego lo ejecuto.
Para poder mostrar por pantalla el resultado, realizo la transferencia de dados de GPU a CPU.

#**2 Armado del Ambiente**
Instala en el cuaderno el modulo CUDA de Python

In [ ]:
!pip install pycuda

#**3 Desarrollo**

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cant =   8#@param {type: "number"}

# --------------------------------------------
from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

vec1_cpu = numpy.random.randn( cant )
vec1_cpu = vec1_cpu.astype( numpy.int32() )

vec2_cpu = numpy.random.randn(cant)
vec2_cpu = vec2_cpu.astype(numpy.int32())

vec1_gpu = cuda.mem_alloc( vec1_cpu.nbytes )
vec2_gpu = cuda.mem_alloc( vec2_cpu.nbytes )

cuda.memcpy_htod( vec1_gpu, vec1_cpu )
cuda.memcpy_htod( vec2_gpu, vec2_cpu )

print("vector", vec1_cpu)

module = SourceModule("""
__global__ void invertirVector( int n, int *V, int *Y)
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
  if( idx <= n ){
    Y[idx] = V[n-1-idx];
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("invertirVector")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cant+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cant), vec1_gpu, vec2_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( vec2_cpu, vec2_gpu )


print("vector resultado", vec2_cpu)

vector [ 1  0  0  0 -1  0  0 -1]
Thread x:  256 , Bloque x: 1
vector resultado [-1  0  0 -1  0  0  0  1]


# **4 Tabla de Pasos**

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( cant ) | Inicializa los vectoes vec1_cpu y vec2_cpu.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria vec2_gpu a CPU memoria vec2_cpu.
CPU      |  print()               | Informo los resultados.



#**5 Conclusion**

Con la paralelizacion se puede observar que la complejidad computacional es constante[2], lo cual resulta mas optimo a comparacion de la ejecucion secuencial, la cual habia dado O(n). Por otra parte, esta es una solución simple, no aspira a aplicar paralelismo masivo porque el máximo tamaño de bloque es 256 hilos, con lo que ése sería el mayor vector que este
código podría aceptar como entrada.


#**6 Bibliografia**
[1] PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[2] Complejidad Computacional:[PDF](https://www.frlp.utn.edu.ar/materias/algoritmos/GUIACOMPLEJIDADDEALGORITMOS.pdf)
